In [1]:
import re
import pandas as pd
import requests
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
from requests import RequestException
from newspaper import Article, ArticleException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By as by
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
class NewsScraper:
    def __init__(self):
        options = Options()
        options.headless = True
        self.driver = webdriver.Firefox(executable_path='/Users/rajwaghela/Downloads/Softwares/geckodriver', options=options)
        self.category_dict = {}
        self.setup()

    def setup(self):
        self.driver.get(url='https://news.google.com/home?hl=en-US&gl=US&ceid=US%3Aen')
        disable_cookies = WebDriverWait(self.driver, timeout=2).until(
            EC.presence_of_element_located((by.XPATH, '/html/body/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[1]/div/div/button'))
        )
        disable_cookies.click()
        category = self.driver.find_elements_by_xpath('//a[@class="brSCsc"][@role="menuitem"]')
        for cat in category:
            self.category_dict[cat.text] = cat.get_attribute('href')
        self.category_dict = dict(list(self.category_dict.items())[5:])

    def run(self):
        for cat, url in self.category_dict.items():
            print('_____________________________________________________________________________________________________')
            print(f'Category = {cat}')
            self.driver.get(url)
            sleep(3)
            self.process_category(cat)

    def process_category(self, cat):
        tabpanel = self.driver.find_elements_by_xpath('//a[@class="WwrzSb"]')
        href = [h.get_attribute('href') for h in tabpanel]
        org_urls = []
        count = 0 
        print('Abstracting links..')
        for url in href:
            try:
                self.driver.get(url)
                sleep(1.5)
                org_urls.append(self.driver.current_url)
                count += 1
                print(count, end=',')
            except WebDriverException as e:
                print(f'Skipping page due to WebDriverException: {e}')
                continue
        print(f'Total no. of links abstracted ==> {len(org_urls)} ')
        total = len(org_urls)
        self.fetch_articles(org_urls, cat, total)

    def fetch_articles(self, urls, cat, total):
        df = {
            'Authors':[],
            'Pubslish_Date':[],
            'Title':[],
            'Text':[],
            'Image':[],
            'Link':[],
            'Category':[]
        }
        count = 0
        count_1 = 0
        count_2 = 0
        headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
        print('Scraping Articles..')
        for url in urls:
            try:
                respones = requests.get(url, headers = headers)
                if respones.status_code != 403 or respones.status_code!=404 or respones.status_code!=419 or respones.status_code!=408 or respones.status_code!=406 or respones.status_code!=400 or respones.status_code!=401 or respones.status_code!=203 or respones.status_code!=204 or respones.status_code!=500 or respones.status_code!=502 or respones.status_code!=503 or respones.status_code!=504 :
                    if ('Youtube'in url) or ('Yahoo' in url):
                        count_1 += 1
                    else:
                        page = Article(url)
                        page.download()
                        page.parse()
                        df['Authors'].append(page.authors)
                        df['Pubslish_Date'].append(page.publish_date)
                        df['Title'].append(page.title)
                        df['Text'].append(page.text)
                        df['Image'].append(page.top_image)
                        df['Link'].append(page.canonical_link)
                        df['Category'].append(cat)
                        count+=1
                        print(count, end=',')
                else:
                    count_2 += 1
            except (ArticleException, RequestException) as e:
                print(f'\nError while processing {url}:{e}')
                continue
        print(f'\nNo. of non articles links == >{count_1}\nNo. of Articles Forbid access ==> {count_2}\nRetrived Articles ==> {total- count_1 - count_2}')

        file = pd.DataFrame(df)
        file.to_csv(f'Datasets1/{cat}.csv', index=False, header=False, mode='a')
        print(f'category {cat} fetched\ndata imported to path "Datasets/{cat}.csv" ')

if __name__ == "__main__":
    scraper = NewsScraper()
    scraper.run()

_____________________________________________________________________________________________________
Category = World
Abstracting links..
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,